Installing sodapy and pandas

In [1]:
%%capture
!pip install sodapy pandas numpy

Available columns:
['address_type', 'agency', 'agency_name', 'bbl', 'borough', 'city', 'closed_date', 'community_board', 'complaint_type', 'created_date', 'cross_street_1', 'cross_street_2', 'descriptor', 'due_date', 'facility_type', 'incident_address', 'incident_zip', 'intersection_street_1', 'intersection_street_2', 'latitude', 'location', 'location_type', 'longitude', 'open_data_channel_type', 'park_borough', 'park_facility_name', 'resolution_action_updated_date', 'resolution_description', 'status', 'street_name', 'unique_key', 'x_coordinate_state_plane', 'y_coordinate_state_plane']

In [7]:
import pandas as pd
import numpy as np
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# First 50000 results, recieved as JSON & returned as dict
# Columns converted to snake case, special chars removed,
# dates and location formatted

results = client.get("fhrw-4uyv", limit=1)

# Convert to pandas DataFrame
records = pd.DataFrame.from_records(results)

#print example record
print(records)

  address_type agency                                        agency_name  \
0      ADDRESS    HPD  Department of Housing Preservation and Develop...   

          bbl   borough      city              closed_date community_board  \
0  3019570030  BROOKLYN  BROOKLYN  2017-01-11T21:02:00.000     02 BROOKLYN   

   complaint_type             created_date           ...             \
0  HEAT/HOT WATER  2017-01-10T11:00:43.000           ...              

  open_data_channel_type park_borough park_facility_name  \
0                 ONLINE     BROOKLYN        Unspecified   

  resolution_action_updated_date  \
0        2017-01-11T21:02:00.000   

                              resolution_description  status    street_name  \
0  The complaint you filed is a duplicate of a co...  Closed  GREENE AVENUE   

  unique_key x_coordinate_state_plane y_coordinate_state_plane  
0   35220081                   992382                   189258  

[1 rows x 28 columns]


### Finding most common complains for 2017 year

In [24]:
results = client.get("fhrw-4uyv",
                     where="created_date BETWEEN '2017-01-01T00:00:00.000' and '2018-01-01T00:00:00.000'",
                     group="complaint_type",
                     select="complaint_type, count(complaint_type)")


# Convert to pandas DataFrame
records = pd.DataFrame.from_records(results)
records.count_complaint_type = records.count_complaint_type.astype(int)
print(records.sort_values('count_complaint_type', ascending=False)[:10])

              complaint_type  count_complaint_type
174      Noise - Residential                230152
163           HEAT/HOT WATER                213521
164          Illegal Parking                146122
203         Blocked Driveway                136097
125         Street Condition                 93265
149   Street Light Condition                 84195
87      UNSANITARY CONDITION                 79282
103  Noise - Street/Sidewalk                 73085
2               Water System                 65101
106                    Noise                 60171


### Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

In [34]:
ten_common_overall_complains = records.sort_values('count_complaint_type', ascending=False)[:10]

where_clause = "(complaint_type = '{}'".format(ten_common_overall_complains.iloc[0]['complaint_type'])
for complaint_type in ten_common_overall_complains['complaint_type'][1:]:
    where_clause += " OR complaint_type = '{}'".format(complaint_type)
where_clause += ")"

boroughs = client.get("fhrw-4uyv",
                     where="{} AND created_date BETWEEN '2017-01-01T00:00:00.000' and '2018-01-01T00:00:00.000'".format(where_clause),
                     group="borough,complaint_type",
                     select="borough, complaint_type, count(complaint_type)")
boroughs = pd.DataFrame.from_records(boroughs)

print(boroughs.sort_values(['borough', 'complaint_type']))

          borough           complaint_type count_complaint_type
26          BRONX         Blocked Driveway                24574
11          BRONX           HEAT/HOT WATER                68718
42          BRONX          Illegal Parking                16122
21          BRONX                    Noise                 3134
18          BRONX      Noise - Residential                57663
13          BRONX  Noise - Street/Sidewalk                14025
17          BRONX         Street Condition                11761
47          BRONX   Street Light Condition                18410
5           BRONX     UNSANITARY CONDITION                24561
56          BRONX             Water System                10221
43       BROOKLYN         Blocked Driveway                49302
55       BROOKLYN           HEAT/HOT WATER                66984
34       BROOKLYN          Illegal Parking                55380
37       BROOKLYN                    Noise                15421
40       BROOKLYN      Noise - Residenti

### Consider only the 10 most common overall complaint types.  For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?

In [53]:
# find most populous zip codes
zip_codes = client.get("fhrw-4uyv",
                     where="created_date BETWEEN '2017-01-01T00:00:00.000' and '2018-01-01T00:00:00.000'",
                     group="incident_zip",
                     select="incident_zip, count(incident_zip)")


# Convert to pandas DataFrame
zip_codes = pd.DataFrame.from_records(zip_codes)
zip_codes.count_incident_zip = zip_codes.count_incident_zip.astype(int)
zip_codes = zip_codes.sort_values('count_incident_zip', ascending=False)[:10]
#print(zip_codes)

where_clause = "(complaint_type = '{}'".format(ten_common_overall_complains.iloc[0]['complaint_type'])
for complaint_type in ten_common_overall_complains['complaint_type'][1:]:
    where_clause += " OR complaint_type = '{}'".format(complaint_type)
where_clause += ")"

for zip_code in zip_codes['incident_zip']:
    #print(zip_code, "*****")
    #print("{} AND incident_zip = '{}' AND created_date BETWEEN '2017-01-01T00:00:00.000' and '2018-01-01T00:00:00.000'".format(where_clause, zip_code))
    incident_zip = client.get("fhrw-4uyv",
                         where="{} AND incident_zip = '{}' AND created_date BETWEEN '2017-01-01T00:00:00.000' and '2018-01-01T00:00:00.000'".format(where_clause, zip_code),
                         group="incident_zip, complaint_type",
                         select="incident_zip, complaint_type, count(complaint_type)")
    incident_zip = pd.DataFrame.from_records(incident_zip)

    print(incident_zip.sort_values(['incident_zip', 'complaint_type']))

            complaint_type count_complaint_type incident_zip
0         Blocked Driveway                 2203        11226
1           HEAT/HOT WATER                 7569        11226
2          Illegal Parking                 1076        11226
3                    Noise                  440        11226
4      Noise - Residential                 4854        11226
5  Noise - Street/Sidewalk                 1831        11226
6         Street Condition                  491        11226
7   Street Light Condition                  202        11226
8     UNSANITARY CONDITION                 3155        11226
9             Water System                  406        11226
            complaint_type count_complaint_type incident_zip
0         Blocked Driveway                 3042        11385
1           HEAT/HOT WATER                 1526        11385
2          Illegal Parking                 4135        11385
3                    Noise                  507        11385
4      Noise - Residenti

### Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017? 

In [91]:
results = client.get("fhrw-4uyv",
                     where="created_date BETWEEN '2017-01-01T00:00:00.000' and '2018-01-01T00:00:00.000'",
                     group="borough",
                     select="borough, count(complaint_type)")


# Convert to pandas DataFrame
borough_complains = pd.DataFrame.from_records(results)
borough_complains = borough_complains.drop([4])
borough_complains.count_complaint_type = borough_complains.count_complaint_type.astype(int)
borough_complains = borough_complains.sort_values('borough', ascending=False)[:10]
borough_complains = borough_complains.reset_index(drop=True)
#print(borough_complains)

'''
FIPS Codes for NYC boroughs:
005 - Bronx
047 - Kings (Brooklyn)
061 - New York (Manhattan)
081 - Queens
085 - Richmond (Staten Island)

Should be prefixed by 36 for state of NY
36005 - Bronx
36047 - Kings (Brooklyn)
36061 - New York (Manhattan)
36081 - Queens
36085 - Richmond (Staten Island)
'''

borough_complains['fips_code'] = pd.Series([36085, 36081, 36061, 36047, 36005])
#print(borough_complains)

fips_codes = "(fips_code = 36085"
for code in [36081, 36061, 36047, 36005]:
    fips_codes += " OR fips_code = {}".format(code)
fips_codes +=")"
#print(fips_codes)
pop_client = Socrata("data.ny.gov", None)
results = pop_client.get("9xc5-th43",
                     where="year = 2010 AND program_type = 'Census Base Population' AND {}".format(fips_codes),
                     select="fips_code, population")

# Convert to pandas DataFrame
borough_population = pd.DataFrame.from_records(results)
borough_population.population = borough_population.population.astype(int)
borough_population.fips_code = borough_population.fips_code.astype(int)
#print(borough_population)
brough_merged = pd.merge(borough_complains, borough_population, on = 'fips_code')
#print(brough_merged)

brough_merged['relative'] = brough_merged['population'] / brough_merged['count_complaint_type']
print(brough_merged.sort_values('relative', ascending=False))

         borough  count_complaint_type  fips_code  population  relative
1         QUEENS                586925      36081     2230545  3.800392
0  STATEN ISLAND                126720      36085      468730  3.698943
2      MANHATTAN                480304      36061     1586184  3.302458
3       BROOKLYN                760970      36047     2504706  3.291465
4          BRONX                449056      36005     1384794  3.083789


So, Queens borough is biggest complainer relativaly to population.